케라스에서 콜백은 학습중에 호출되는 유틸리티들을 말합니다.  
모듈 [tf.keras.callbacks](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks)에서 목록을 확인할 수 있습니다.  
이중에서 우리는 ModelCheckPoint, EarlyStopping, LearningRateScheduler, TensorBoard를 실습해 보겠습니다.

---
신경망 초기화를 반복해야 되서 신경망을 생성하는 함수를 만들어 놓겠습니다.  
데이터는 MNIST를 사용하고 훈련데이터중 첫 10,000장은 검증용으로 나머지 50,000장은 훈련용으로 사용하겠습니다.

In [1]:
from tensorflow import keras
from keras.layers import Dense, Dropout
from keras.datasets import mnist

(images, labels), (test_images, test_labels) = mnist.load_data()
images = images.reshape((60000, 28 * 28)).astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28)).astype("float32") / 255
train_images, val_images = images[10000:], images[:10000]
train_labels, val_labels = labels[10000:], labels[:10000]

def get_mnist_model():
    inputs = keras.Input(shape=(28 * 28,))
    features = Dense(512, activation="relu")(inputs)
    features = Dropout(0.5)(features)
    outputs = Dense(10, activation="softmax")(features)
    model = keras.Model(inputs, outputs)
    return model

model = get_mnist_model()
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 407,050 (1.55 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

# 자동저장

클래스 [tf.keras.callbacks.ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint)를 이용해 학습중에 모델을 자동저장할 수 있습니다.
- `filepath="check_point.h5"` : 현재 작업 디렉토리에 check_point.h5란 파일명으로 저장합니다.
- `monitor="val_loss"` : 검증데이터의 손실함수 값을 모니터링합니다.
- `save_best_only=True` 에폭마다 무조건 덮어쓰는게 아니라 모니터링 값이 전 에폭보다 더 안좋으면 덮어쓰지 않습니다.

인스턴스를 [fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) 메서드에 인수로 지정해줍니다.  
현재 작업 디렉토리를 열어 보면 파일 check_point.keras를 확인할 수 있습니다.   
코랩은 구글 드라이브가 현재 작업 디렉토리가 아닙니다.  
왼쪽에 파일 아이콘을 클릭해보세요.  
출력에서 val_loss를 보면 몇 에폭에 저장됬는지 알수 있습니다.

In [2]:
from keras.callbacks import ModelCheckpoint

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

check_point = ModelCheckpoint(filepath="check_point.keras",
                                 monitor="val_loss",
                                 save_best_only=True)

model.fit(train_images, train_labels,
          epochs=10,
          callbacks=check_point,
          validation_data=(val_images, val_labels))

Epoch 1/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.8652 - loss: 0.4483 - val_accuracy: 0.9579 - val_loss: 0.1446
Epoch 2/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9514 - loss: 0.1652 - val_accuracy: 0.9657 - val_loss: 0.1180
Epoch 3/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9628 - loss: 0.1337 - val_accuracy: 0.9719 - val_loss: 0.1065
Epoch 4/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9690 - loss: 0.1104 - val_accuracy: 0.9758 - val_loss: 0.0963
Epoch 5/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9728 - loss: 0.0994 - val_accuracy: 0.9743 - val_loss: 0.0993
Epoch 6/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9746 - loss: 0.0922 - val_accuracy: 0.9770 - val_loss: 0.0903
Epoch 7/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9761 - loss: 0.0856 - val_accuracy: 0.9795 - val_loss: 0.0906
Epoch 8/10
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9775 - loss: 0.0827 - 

[tf.keras.models.load_model](https://www.tensorflow.org/api_docs/python/tf/keras/models/load_model)로 저장한 파일에서 모델을 불러올 수 있습니다.

In [3]:
from keras import models

loaded_model = models.load_model("check_point.keras")
loaded_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 814,102 (3.11 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 407,052 (1.55 MB)

**[실습1] (5분) 불러온 모델의 검증 데이터에 대한 정확도를 출력하시오. 저장한 에폭의 정확도와 일치하는지 확인하시오.**

In [4]:
loaded_model.evaluate(val_images,val_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.9778 - loss: 0.0942


[0.09028729051351547, 0.9769999980926514]

# 학습 조기종료

과적합에 대항하는 가장 단순한 방법은 과적합이 시작되는 순간 학습을 멈추는 것입니다.  
클래스 [tf.keras.callbacks.EarlyStopping](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping)를 이용해 학습을 조기종료할 수 있습니다.  
- `monitor="val_accuracy"` : 검증 데이터의 정확도를 모니터링합니다.
-  `patience=2` : 모니터링 값이 향상 되지 않는다고 바로 종료하지 않고 2에폭동안 더 지켜보고 향상되지 않으면 학습을 종료합니다.

인스턴스를 [fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) 메서드에 인수로 지정해줍니다.

In [8]:
from keras.callbacks import EarlyStopping

model = get_mnist_model()

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

check_point = ModelCheckpoint(filepath="check_point1.keras",
                                 monitor="val_accuracy",
                                 save_best_only=True)

early_stopping = EarlyStopping(monitor="val_accuracy",
                               patience=3)

model.fit(train_images, train_labels,
          epochs=20,
          callbacks=[early_stopping,check_point],
          validation_data=(val_images, val_labels))

Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8641 - loss: 0.4499 - val_accuracy: 0.9575 - val_loss: 0.1487
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9507 - loss: 0.1642 - val_accuracy: 0.9685 - val_loss: 0.1124
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9624 - loss: 0.1271 - val_accuracy: 0.9745 - val_loss: 0.0938
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9679 - loss: 0.1163 - val_accuracy: 0.9723 - val_loss: 0.1061
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9727 - loss: 0.0981 - val_accuracy: 0.9752 - val_loss: 0.1026
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9740 - loss: 0.0934 - val_accuracy: 0.9776 - val_loss: 0.0908
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9754 - loss: 0.0866 - val_accuracy: 0.9786 - val_loss: 0.0915
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9776 - loss: 0.0815 - 

In [9]:
loaded_model1 = models.load_model("check_point1.keras")
loaded_model1.summary()

Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 512)            │       401,920 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 814,102 (3.11 MB)

 Trainable params: 407,050 (1.55 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 407,052 (1.55 MB)

In [10]:
loaded_model1.evaluate(val_images,val_labels)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9807 - loss: 0.0943


[0.09326817095279694, 0.9817000031471252]

모델이 학습하는 동안 통상적으로 검증데이터의 정확도는 증감을 반복합니다.  
살짝 감소하였다고 바로 조기종료해버리면 충분한 학습이 이루어지지 않을 수 있습니다.  

In [ ]:
model = get_mnist_model()

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

early_stopping = EarlyStopping(monitor="val_accuracy")

model.fit(train_images, train_labels,
          epochs=15,
          callbacks=early_stopping,
          validation_data=(val_images, val_labels))

**[실습2] (5분) 검증 데이터의 손실함수 값이 3에폭동안 향상이 없으면 조기종료되도록 콜백을 설정후 학습시키시오.**

In [ ]:
model = get_mnist_model()

model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

early_stopping = EarlyStopping(monitor="val_accuracy")

model.fit(train_images, train_labels,
          epochs=15,
          callbacks=early_stopping,
          validation_data=(val_images, val_labels))

# 학습률 스케줄러

클래스 [tf.keras.callbacks.LearningRateScheduler](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/LearningRateScheduler)를 이용해 학습중에 학습률을 변경할 수 있습니다.  
처음에는 성큼성큼 걷다가 나중에 잰 걸음으로 걷는 방법이 효율적일 겁니다.  
AdaGrad이나 RMSprop과 비슷한 맥락입니다.  
먼저 에폭별 학습률을 리턴하는 함수를 정의합니다.  
10에폭까지는 1의 학습률로 학습하다가 11에폭부터는 이전 에폭 학습률에 $e^{-0.1}$($\approx 0.9048$)을 곱한 학습률로 학습합니다.  
코드에서 epoch는 0부터 시작합니다.  
인스턴스를 [fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) 메서드에 인수로 지정해줍니다.  
에폭마다 학습률이 맨 오른쪽에 출력됩니다.

In [11]:
import numpy as np
from keras.callbacks import LearningRateScheduler

model = get_mnist_model()

optimizer = keras.optimizers.SGD(learning_rate=1.)
model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

def scheduler(epoch, learning_rate):
    if epoch<10:
        return learning_rate
    else:
        return learning_rate*np.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

model.fit(train_images, train_labels,
          epochs=15,
          callbacks=lr_scheduler)

Epoch 1/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6164 - loss: 1.4094 - learning_rate: 1.0000
Epoch 2/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8161 - loss: 0.6772 - learning_rate: 1.0000
Epoch 3/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8377 - loss: 0.6275 - learning_rate: 1.0000
Epoch 4/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8437 - loss: 0.6249 - learning_rate: 1.0000
Epoch 5/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8531 - loss: 0.5920 - learning_rate: 1.0000
Epoch 6/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8640 - loss: 0.5590 - learning_rate: 1.0000
Epoch 7/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8740 - loss: 0.5048 - learning_rate: 1.0000
Epoch 8/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8794 - loss: 0.4957 - learning_rate: 1.0000
Epoch 9/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8780 - loss: 0.5011 - learning_rate:

**[실습3] (10분) $n$번째 에폭에서의 학습률이 $e^{-0.2(n-1)}$이 되도록 콜백을 설정하고 학습시키시오.**

In [18]:
model = get_mnist_model()

optimizer = keras.optimizers.SGD(learning_rate=1.)


model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics = ['accuracy'])

def schedule(epoch,learning_rate):
    return np.exp(-0.2*epoch)

lr_scheduler = LearningRateScheduler(schedule)

model.fit(train_images,train_labels,
          epochs=15,
          callbacks=lr_scheduler)

Epoch 1/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.6048 - loss: 1.4068 - learning_rate: 1.0000
Epoch 2/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8357 - loss: 0.6007 - learning_rate: 0.8187
Epoch 3/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8778 - loss: 0.4550 - learning_rate: 0.6703
Epoch 4/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9017 - loss: 0.3665 - learning_rate: 0.5488
Epoch 5/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9159 - loss: 0.2983 - learning_rate: 0.4493
Epoch 6/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9243 - loss: 0.2666 - learning_rate: 0.3679
Epoch 7/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9321 - loss: 0.2333 - learning_rate: 0.3012
Epoch 8/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9393 - loss: 0.2083 - learning_rate: 0.2466
Epoch 9/15
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9448 - loss: 0.1891 - learning_rate:

# 텐서보드

클래스 [tf.keras.callbacks.TensorBoard](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/TensorBoard)를 이용해 학습 히스토리와 계산 그래프를 시각화 할수 있습니다.  
 - `log_dir="./tensorboard"` : 로그 파일을 저장할 디렉토리를 지정합니다.

인스턴스를 [fit](https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit) 메서드에 인수로 지정해줍니다.  
폴더를 열어보면 현재 작업 디렉토리 밑에 tensorboard란 디렉토리가 새로 생겼을겁니다.  
그 밑에 train 디렉토리와 validation 디렉토리가 있습니다.  
여기에 훈련로그와 검증로그가 저장됩니다.

In [23]:
from keras.callbacks import TensorBoard

model = get_mnist_model()
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

tensorboard = TensorBoard(log_dir="/tensorboard")
model.fit(train_images, train_labels,
          epochs=20,
          validation_data=(val_images, val_labels),
          callbacks=[tensorboard])

Epoch 1/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8637 - loss: 0.4552 - val_accuracy: 0.9561 - val_loss: 0.1482
Epoch 2/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9513 - loss: 0.1664 - val_accuracy: 0.9650 - val_loss: 0.1195
Epoch 3/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9631 - loss: 0.1304 - val_accuracy: 0.9718 - val_loss: 0.1019
Epoch 4/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9670 - loss: 0.1161 - val_accuracy: 0.9737 - val_loss: 0.0987
Epoch 5/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9730 - loss: 0.0965 - val_accuracy: 0.9768 - val_loss: 0.0960
Epoch 6/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.9732 - loss: 0.0962 - val_accuracy: 0.9750 - val_loss: 0.0979
Epoch 7/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9767 - loss: 0.0850 - val_accuracy: 0.9774 - val_loss: 0.0925
Epoch 8/20
1563/1563 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9787 - loss: 0.0832 - 

매직 명령어를 써서 텐서보드를 불러옵니다.  
텐서보드는 로그 디렉토리에서 로그 기록을 불러와 시각화 합니다.  
여태 matplotlib으로 직접 그렸던 정확도와 손실함수의 그래프가 자동으로 출력되서 나오네요.  
GRAPHS탭을 클릭하시면 신경망의 계산그래프가 시각화되서 출력됩니다.  
이렇게 노트북 안에서 볼수도 있지만 로컬 환경이라면 다른 탭을 열고 주소창에 http://localhost:6006 을 입력하면 꽉 찬 화면으로 볼수있습니다.

In [26]:
%load_ext tensorboard
%tensorboard --logdir /tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6007 (pid 2760), started 0:00:05 ago. (Use '!kill 2760' to kill it.)

**[실습4] (2분) 정확도와 손실함수의 그래프에 마우스를 대보시오. 계산그래프의 노드를 더블 클릭해보시오.**

콜백 인스턴스들을 리스트로 묶은후 fit메서드에서 인수로 지정하면 여러개의 콜백을 동시에 사용할 수 있습니다.

In [ ]:
model = get_mnist_model()

optimizer = keras.optimizers.SGD(learning_rate=1.)

model.compile(optimizer=optimizer,
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

check_point = ModelCheckpoint(filepath="check_point.keras",
                                 monitor="val_loss",
                                 save_best_only=True)

early_stopping = EarlyStopping(monitor="val_accuracy",
                               patience=2)

def scheduler(epoch, learning_rate):
    if epoch<10:
        return learning_rate
    else:
        return learning_rate*np.exp(-0.1)

lr_scheduler = LearningRateScheduler(scheduler)

tensorboard = TensorBoard(log_dir="./tensorboard")

model.fit(train_images, train_labels,
          epochs=20,
          callbacks=[check_point, early_stopping, lr_scheduler, tensorboard],
          validation_data=(val_images, val_labels))